In [1]:
import logging
import warnings

from hydra import compose, initialize
from omegaconf import OmegaConf

from retailsynth.datasets.complete_journey.preprocess_pipeline import (
    run_preprocess,
    PreprocessPipeline,
)
from retailsynth.base_config import load_config_store

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

warnings.filterwarnings("ignore", category=FutureWarning)


## Introduction

This notebook provides a deep dive exploratory analysis and documentation of the data preprocessing and cleaning steps
required to leverage The Complete Journey dataset.[1] We note that several demand prediction papers[2][3] have used this
same dataset and provide additional insights on the data quality beyond what is presented here.


## Data loading

We begin by instantiating the pipeline and downloading the raw data. We use three main tables: transactions, customer
demographics and product information. For more information about this data source, please refer to the Dunnhumby
website.


In [2]:
with initialize(version_base=None, config_path="cfg"):
    load_config_store()
    cfg = compose(config_name="real_dataset")
    cfg = OmegaConf.to_object(cfg)

preprocess_pipeline = PreprocessPipeline(
    raw_data_config=cfg.raw_data,
)

In [3]:
customers = preprocess_pipeline.dataset.customers
transactions = preprocess_pipeline.dataset.transactions
products = preprocess_pipeline.dataset.products


## Data cleaning

In this section, we will describe the contents of each table and the cleaning steps we implemented to mitigate data
quality issues.


### Customer table cleaning

The `customers` table holds demographic attributes for each customer. Here is a sample of the data:

In [4]:
customers.set_index("customer_key").head(2)

,age,income,home_ownership,marital_status,household_size,household_comp,kids_count
customer_key,,,,,,,
1,65+,35-49K,Homeowner,Married,2,2 Adults No Kids,0
1001,45-54,50-74K,Homeowner,Unmarried,1,1 Adult No Kids,0


However, in the transactions table there are orders attributed to customers customers not recorded in the demographic
table. With `record_unrecognized_customer`, we add these customers to the customer table.

In [5]:
# complete list customers in our record
recorded_customers = customers.customer_key.unique()
# transactions from customer not in our record
print(
    "Number of transactions from unrecognizable customers: ",
    transactions[~transactions.customer_key.isin(recorded_customers)].shape[0],
)
_ = preprocess_pipeline.record_unrecognized_customer()

2024-03-05 20:55:09,610 - INFO - There are unrecognized customer ids in the transactions table. Number of unrecognized customer ids: 1668


Number of transactions from unrecognizable customers:  640457



### Product table cleaning

The `products` table contains information about products, where the `product_nbr` column serves as the unique
identifier. The table provides information, including product department, category description, subcategory description,
manufacturer, brand, and package size.

There are groups of products that share the same attributes but have different product IDs. These are likely very
similar from the customers' perspective. We show an example below:

In [6]:
package_size = "13.7 OZ"
manufacturer_id = 6380
subcategory_desc = "TOMATOES: STEWED/DICED/CRMD"

products[
    (products.package_size == package_size)
    & (products.manufacturer_id == manufacturer_id)
    & (products.subcategory_desc == subcategory_desc)
]

,product_nbr,manufacturer_id,department,brand,category_desc,subcategory_desc,package_size
86082,15449249,6380,GROCERY,National,VEGETABLES - SHELF STABLE,TOMATOES: STEWED/DICED/CRMD,13.7 OZ
86083,15449250,6380,GROCERY,National,VEGETABLES - SHELF STABLE,TOMATOES: STEWED/DICED/CRMD,13.7 OZ
86090,15449588,6380,GROCERY,National,VEGETABLES - SHELF STABLE,TOMATOES: STEWED/DICED/CRMD,13.7 OZ


As the digital twin simulation focuses on the customer perspective, we overwrite the duplicate `product_nbrs` with the
first unique found in the product table.

In [7]:
_ = preprocess_pipeline.drop_duplicate_product_id()

2024-03-05 20:55:10,303 - INFO - Use the same label for products with the same hierarchy information. Number of products are decreased to 32333.


In [8]:
products = preprocess_pipeline.dataset.products
products[
    (products.package_size == package_size.lower())
    & (products.manufacturer_id == manufacturer_id)
    & (products.subcategory_desc == subcategory_desc.lower())
]

,manufacturer_id,department,brand,category_desc,subcategory_desc,package_size,product_nbr,all
86082,6380,grocery,national,vegetables - shelf stable,tomatoes: stewed/diced/crmd,13.7 oz,30101,all



### Transaction table cleaning

The `transactions` table contains records of each customer's purchase behavior, providing information such as
transaction date, discount policies, quantity sold, and more. The original dataset includes three sources of discount:
retail discount (from a loyalty card program), coupon discount (supplied by the manufacturer), and coupon match
discount (supplied by the retailer's match of the manufacturer coupon).

In [9]:
transactions.head(5)

,customer_key,store_nbr,basket_id,product_nbr,item_qty,sales_amt,retail_disc,coupon_disc,coupon_match_disc,week,transaction_timestamp,day
0,900,330,31198570044,1095275,1,0.50,-0.00,-0.0,-0.0,1,2017-01-01 11:53:26,1
1,900,330,31198570047,9878513,1,0.99,-0.10,-0.0,-0.0,1,2017-01-01 12:10:28,1
2,1228,406,31198655051,1041453,1,1.43,-0.15,-0.0,-0.0,1,2017-01-01 12:26:30,1
3,906,319,31198705046,1020156,1,1.50,-0.29,-0.0,-0.0,1,2017-01-01 12:30:27,1
4,906,319,31198705046,1053875,2,2.78,-0.80,-0.0,-0.0,1,2017-01-01 12:30:27,1



#### 1. Product price calculation

The original dataset does not provide the discount portion. Thus we implement a helper method to compute the actual
dealt price, which includes unit price (after applying all discounts), and empirical discount percentage.

In [10]:
_ = preprocess_pipeline.add_pricing_columns()
transactions = preprocess_pipeline.dataset.transactions
columns_in_use = [
    "customer_key",
    "product_nbr",
    "week",
    "item_qty",
    "sales_amt",
    "unit_price",
    "discount_portion",
]
transactions.loc[:, columns_in_use].head(5)

,customer_key,product_nbr,week,item_qty,sales_amt,unit_price,discount_portion
0,1085,15,1,1,1.79,1.79,0.067708
1,1085,32,1,1,1.00,1.00,0.371069
2,1085,110,1,1,1.50,1.50,0.462366
3,1085,195,1,1,1.00,1.00,0.224806
4,1085,318,1,1,2.49,2.49,0.000000



#### 2. Invalid transactions

To ensure that every row in the `transactions` table contains meaningful data, we need to inspect the values in each
column to determine if they are interpretable in the context of a purchase event. However, we have observed transactions
with negative quantity sold and negative money spent. These records likely represent product returns instead of actual
purchases or valid store visits.

In [11]:
transactions[(transactions.item_qty <= 0) | (transactions.sales_amt <= 0)].head(2)

,basket_id,product_nbr,customer_key,week,day,item_qty,sales_amt,unit_price,discount_portion
841,31198566701,13144,80,1,1,0,0.0,NaN,NaN
1167,31198635234,1847,1715,1,1,0,0.0,NaN,NaN


We use the following method to filter out these transactions and remove customer and product records that are not
associated with any valid transactions.

In [12]:
_ = preprocess_pipeline.clean_transactions()


### Putting it all together


We have encapsulated all filters introduced in this method into a helper method called `run_preprocess`. The method
signature is shown below.

In [13]:
help(run_preprocess)

Help on function run_preprocess in module retailsynth.datasets.complete_journey.preprocess_pipeline:

run_preprocess(config: retailsynth.base_config.Config)
    Run the preprocess pipeline.
    
    This function applies the preprocess pipeline to the raw data based on the provided configuration.
    
    Parameters
    ----------
    config : Config
        Configuration for the whole run.
    
    Returns
    -------
    tuple
        A tuple containing the preprocessed customers, products, and transactions dataframes.



The following codes run all preprocessing steps described above, and outputs the customer demographic table, the product
hierarchy table, and the transaction table.

In [14]:
customers, products, transactions = run_preprocess(cfg)

2024-03-05 20:55:10,857 - INFO - Start preprocess pipeline
2024-03-05 20:55:11,761 - INFO - Filter out transactions with non-positive quantity sold or money spent. Number of transactions are decreased to 1458032.
2024-03-05 20:55:12,442 - INFO - Use the same label for products with the same hierarchy information. Number of products are decreased to 32333.
2024-03-05 20:55:12,450 - INFO - Added category numbers.
2024-03-05 20:55:12,497 - INFO - There are unrecognized customer ids in the transactions table. Number of unrecognized customer ids: 1668
2024-03-05 20:55:13,424 - INFO - Preprocess pipeline finished
2024-03-05 20:55:13,424 - INFO - Number of customers: 2469
2024-03-05 20:55:13,424 - INFO - Number of products: 26176
2024-03-05 20:55:13,424 - INFO - Number of transactions: 1299274
2024-03-05 20:55:13,425 - INFO - Annotate products
2024-03-05 20:55:13,448 - INFO - Sampling dataset
2024-03-05 20:55:13,448 - INFO - Number of customers: 100
2024-03-05 20:55:13,449 - INFO - Number of 


## References

1. Dunnhumby Source Files. https://www.dunnhumby.com/source-files/
2. Maasakkers, et al. Next-basket prediction in a high-dimensional setting using gated recurrent units, Expert Systems
   with Applications, Volume 212, 2023, 118795, ISSN 0957-4174. https://doi.org/10.1016/j.eswa.2022.118795.
3. Ariannezhad, Mozhdeh, et al. "ReCANet: A Repeat Consumption-Aware Neural Network for Next Basket Recommendation in
   Grocery Shopping." Proceedings of the 45th International ACM SIGIR Conference on Research and Development in
   Information Retrieval. 2022.